В данной статье исследуется зависимость качества обучения нейронных сетей от стратегии изменения learning rate.

Проблемой данного исследования, однако, является очень низкое разнообразие исследованных датасетов и архитектур сетей. Использовались только задачи классификации изображений, датасеты MNIST и CIFAR-10, имеющие примерно одинаковое количество примеров, только сверточные сети, только оптимизатор SGD с моментом и метод Нестерова, и только обучение "с нуля" (не файн-тюнинг).

### Фиксированный learning rate

Сначала авторы оценивают наилучший фиксированный LR на датасете CIFAR-10, с оптимизатором SGD и простой сверточной сетью из 3 сверточных слоев, взятой из официальных примеров в документации библиотеки Caffe (авторы называют эту архитектуру CNN3).

На графике ниже каждая позиция по оси Learning Rate соответствует одной сети, обучаемой с данным Learning Rate. Показана точность после 2, 20, 40, 80, 140 эпох.

<img src="assets/demyst1.jpg" width="400" align="center">

По-видимому, данный эксперимент авторы рассматривают как предварительный, чтобы использовать полученные значения как "отправную точку" для подбора параметров в стратегиях изменения LR и пока не делают никаких выводов. Здесь также можно вспомнить метод "LR range test" из работы [Cyclical Learning Rates for Training Neural Networks]($Cyclical Learning Rates for Training Neural Networks$) ([lr_finder](https://fastai1.fast.ai/callbacks.lr_finder.html) в библиотеке fast.ai). Однако такой метод может не сработать для некоторых типов сетей, которые обучаются очень медленно (например, сети с сигмоидой или с высоким dropout).

### Циклический learning rate

Один из простых вариантов циклического LR можно описать следующей формулой: $|\arcsin(\sin x)|$. Более сложный, названный авторами TRI2, описывается такой формулой ($t$ - номер шага, $l$, $k_0$, $k_1$ - гиперпараметры; $k_0$ - минимальный LR, $k_1$ - максимальный LR, $l$ - масштаб по оси времени):

$\textit{learning_rate} = k_0 + (k_1 - k_0)*\cfrac{1}{2^{\text{floor}(t / 2l)}}*\cfrac{2}{\pi}*|\arcsin(\sin \frac{\pi t}{2l})|$

Оба варианта показаны на графиках ниже:

<img src="assets/demyst2.jpg" width="700" align="center">

Фактически весь данный раздел сводится к тому, что авторы подбирают оптимальные параметры $k_0$ и $k_1$.

### Ступенчатое затухание learning rate

Эта стратегия является очень популярной в задачах компьютерного зрения. Каждыe N шагов LR умножается на коэффициент $\alpha$. В данной работе авторы подбирают $N$, фиксируя $\alpha$ в значении 0.85. Мне кажется это довольно странным, потому что чем ближе $\alpha$ к единице, тем сильнее ступенчатое затухание становится похожим на экспоненциальное затухание. Особенность именно ступенчатого затухания в маленьком $\alpha$. В статьях, в которых сети обучаются на ImageNet, часто берут $\alpha = 0.1$.

Авторы подбирают параметры и находят оптимальный $N$, обучая сеть CNN3 на CIFAR-10.

### Стратегия "reduce LR on plateau"

Это еще одна популярная стратегия (авторы называют ее NSTEP), в которой умножение на $\alpha$ происходит тогда, когда начинается стагнация метрики качества на валидации. Авторы также подбирают параметры для конкретной задачи и отмечают, что важным является указание нижней границы LR, иначе в итоге LR станет слишком близким к нулю, и сеть более не сможет обучаться. Конечно, то же самое касается и экспоненциального затухания и (в меньшей степени) полиномиального затухания.

### LRBench

Далее всю собранную экспериментальную информацию авторы неким (не до конца понятным) способом используют в инструменте для автоматического подбора гиперпараметров [LRBench](https://github.com/git-disl/LRBench), который можно использовать и на других задачах.

<img src="assets/demyst3.jpg" width="500" align="center">

С помощью LRBench делается несколько интересных выводов:

1. На MNIST циклический LR приводит к меньшей разнице между loss на обучении и валидации, чем LR с затуханием.
2. На CIFAR-10 циклический LR со стратегией TRI2 опережает по качеству оригинальный циклический LR, предложенный в статье [Cyclical Learning Rates for Training Neural Networks]($Cyclical Learning Rates for Training Neural Networks$), а также стратегию "reduce LR on plateau".

Кроме того в экспериментах авторы оценивают, помимо accuracy, еще несколько метрик:
1. [Top-5 accuracy](https://stats.stackexchange.com/questions/156471/imagenet-what-is-top-1-and-top-5-error-rate)
2. Average confidence: средняя вероятность верного класса среди тех примеров, в которых этот класс предсказан верно.
3. Confidence deviation: среднеквадратичное отклонение вероятности верного класса среди тех примеров, в которых этот класс предсказан верно.
4. Confidence deviation across classes

Однако в работе нет строгого обоснования того, что эти метрики имеет смысл повышать. Top-5 accuracy кажется совсем неподходящей для датасетов MNIST и CIFAR-10, в которых всего по 10 классов. Вероятности, присвоенные верно угаданным классам, возможно и имеют какое-то значение, но это нужно не постулировать, а обосновывать. Здесь есть и контраргумент: "идеальная модель" не всегда должна выдавать 100% вероятность для верного класса, так как бывают и сложные случаи. И кроме того, метрику average confidence можно повысить просто умножив logit'ы на некую константу больше 1.

### Заключение

В целом исследование выглядит достаточно поверхностным. Подбирать параметры - дело очень нехитрое, но гораздо интереснее было отыскать общие закономерности, оббщающиеся на множество разных задач и архитектур. Для этого нужно проводить эксперименты с большим количеством задач и архитектур.

Одним из возможных путей отыскания общих закономерностей, на мой взгляд, может стать применение машинного обучения для предсказания точности по параметрам. В качестве входных данных для алгоритма используются гиперпараметры стратегии LR, а также номер задачи и архитектуры, закодированный one-hot кодированием. Обучаясь, алгоритм сможет отыскать закономерности, связывающие точность и гиперпараметры назависимо от задачи. Для оценки качества самого алгоритма предсказания точности по параметрам, можно разделить все экспериментальные данные на обучающие и тестовые, так чтобы тестовыми оказались крайние значения гиперпараметров. Тогда мы оценим, насколько хорошо наш алгоритм умеет экстраполировать. Высокое качество экстраполяции может говорить об успешном отыскании общих закономерностей зависимости точности от гиперпараметров.